In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import sys, os, glob
sys.path.append('/code/tools/scoring')
from plif import PlifScore

from ui.file_widgets import DockButtons, PathSelector, InputBoxSet

In [6]:
p = PathSelector(start_dir=os.getcwd())
display(p.accord)

b = DockButtons(path_selector=p)
b.render_buttons()

A Jupyter Widget

A Jupyter Widget

In [15]:
pdb_file = b.pdb_button.description
ligands_file = b.lig_button.description
ref_file = b.ref_button.description

In [7]:
inp = InputBoxSet(tlabel='multiple dir:')
inp.render()

A Jupyter Widget

In [ ]:
if os.path.isdir(inp.inbox.value):
    files = glob.glob(os.path.join(inp.inbox.value, '*.sdf'))
else:
    files = [ligands_file]

plif = PlifScore(protein_pdb=pdb_file, 
                 reference_file=ref_file)

if len(files)==1:
    df = plif.score_conformers(file=ligands_file, write=True)
    
else:
    df_list = []
    for f in files:
        df = plif.score_conformers(file=f, write=True)

In [19]:
df_list

[]